In [ ]:
include("../scripts/tools.jl");
include("../scripts/get_grid.jl");
include("../scripts/get_params.jl");
include("../scripts/disc_solvers.jl");

In [ ]:
dm = load_discrete_model("../data/disc/pantagruel.h5", 1.0);

In [ ]:
gps_coord = [38.27656890118348 -4.965603742225435; 47.97737126959275 6.3032033856999234; 39.74292378550017 21.948260020339305]
coord = alberts_projection(gps_coord[:,[2;1]] ./ (180 / pi) )
id = find_gen(dm, gps_coord, 9.0, scale_factor=1.0)
scatter(dm.coord[dm.idgen,2], dm.coord[dm.idgen,1])
scatter!(dm.coord[dm.idgen[id],2], dm.coord[dm.idgen[id],1])

In [ ]:
Nbus = length(dm.load)
is_producing = dm.gen .> 0
id_gen = dm.idgen[is_producing]
id_load = setdiff(1:Nbus, id_gen)
#is_producing = data["gen"][:,2] .> 0
#id_gen = Int64.(data["gen"][is_producing, 1])
#id_load = setdiff(1:length(l), id_gen)
ng = length(id_gen)
nl = length(id_load)

#println(size(dm.idb))

g = zeros(Nbus)
g[dm.idgen] .= dm.gen
p = -dm.load + g
p .-= sum(p) / Nbus

og = zeros(ng)

edges = Int64.(zeros(size(dm.idb)))
line_start = dm.idb[:, 1]
line_end = dm.idb[:, 2]
# bus reordering: generator buses first 
for i in 1:ng
    edges[line_start .== id_gen[i], 1] .= i
    edges[line_end .== id_gen[i], 2] .= i
end
for i in 1:nl
    edges[line_start .== id_load[i], 1] .= i + ng
    edges[line_end .== id_load[i], 2] .= i + ng
end
nline = size(dm.idb,1)
id = edges
inc = sparse([id[:,1]; id[:,2]], [1:nline; 1:nline], [-ones(nline);ones(nline)])

# set initial conditions
og = zeros(ng)
thg = dm.th[id_gen]
thl = dm.th[id_load]
mg = dm.mg[is_producing]
dg = dm.dg[is_producing]
#dload = vec(data["load_freq_coef"])
dg += dm.load[id_gen]
dl = dm.load[id_load]
pg = p[id_gen]
pl = p[id_load]
p = [pg; pl];

In [ ]:
# get the stable solution
b2 = - im .* dm.bline
Ybus = conj(inc * sparse(1:nline, 1:nline, b2) * inc')
q = zeros(nb);
V = ones(nb);
theta = zeros(nb);

V, theta, iter = NRsolver(Ybus, V, theta, -p, q, Array{Int64,1}([]), 9, tol = 1E-30, maxiter = 10)
thg = theta[1:ng]
thl = theta[ng+1:end];
og = zeros(ng);

In [ ]:
dt = 1e-3
Ndt = 5e4
interval = 50
ts = zeros(Int64.(floor(Ndt / interval))+1)
omegas = zeros(ng, Int64.(floor(Ndt / interval))+1)
k = 1
dp = zeros(ng)
dp[1] = -9.0
for i in 1:Ndt
    y = radau5(og, thg, thl, mg, dg, dl, pg+dp, pl, inc, dm.bline, dt, maxiter = 14, tol = 1E-6)
    if(mod(i, interval) == 0)
        println(i, " / ", Ndt, " (", floor(100*i/Ndt), "%)")
            omegas[:, k] = y[1 : ng]
            ts[k] = i*dt
            k += 1
        end
    og = y[1:ng]
    thg = y[ng + 1 : 2 * ng]
    thl = y[2 * ng + 1 : end]
end
    

In [ ]:
fid = h5open("../numerics/disc_results.h5", "w")
write(fid, "ts", ts)
write(fid, "omegas", omegas)
close(fid)